In [1]:
import sys
sys.path.append('../src/')

import wider_dataset

from benchmark.tools import *
from benchmark.models import Detector
from benchmark.utils import TsvHandle, run_detection, tf_to_image_faces

# https://github.com/serengil/tensorflow-101/blob/master/python/opencv-dnn-face-detection.ipynb
# https://github.com/AyushExel/ml4face-detection/blob/master/Final-Face%20Detection%20Notebook.ipynb

# https://raw.githubusercontent.com/opencv/opencv/4.5.4/samples/dnn/face_detector/deploy.prototxt
# https://github.com/opencv/opencv_3rdparty/raw/dnn_samples_face_detector_20170830/res10_300x300_ssd_iter_140000.caffemodel

In [2]:
train, test, validation = tfds.load('wider_dataset', split=['train', 'test', 'validation'])
data = validation

In [3]:
class ResNetDetector(Detector):
    TARGET_SIZE = (300, 300)
    
    CONF_THRESHOLD = 0.5
    NMS_THRESHOLD = 0.4
    
    def __init__(self):
        self.detector = cv2.dnn.readNetFromCaffe("./resnet/deploy.prototxt.txt", "./resnet/res10_300x300_ssd_iter_140000.caffemodel")
    
    def detect(self, image):
        (h, w) = image.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0,
            (300, 300), (104.0, 177.0, 123.0))

        self.detector.setInput(blob)
        detections = self.detector.forward()
        
        boxes = []
        confidence = []
        
        for i in range(0, detections.shape[2]):
            conf = detections[0, 0, i, 2]
            if conf > 0.5:
                confidence.append(conf)
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                boxes.append([startX, startY, endX, endY])

        return boxes, confidence
    
resnet = ResNetDetector()

In [4]:
with TsvHandle('resnet') as tsv:
    run_detection(tsv, data, resnet, cv2.COLOR_RGB2BGR, False, False)

Running detection
[====================] 100% [3226/3226]